In [36]:
import pandas as pd
tot_data = pd.read_csv('movies_data.csv')

In [89]:
tot_data.head(5)

,id,genres,vote_average,vote_count,popularity,title,belongs_to_collection,overview
0,862,Animation Comedy Family,7.7,5415.0,21.9469,Toy Story,"{'id': 10194, 'name': 'Toy Story Collection', ...","Led by Woody, Andy's toys live happily in his ..."
1,8844,Adventure Fantasy Family,6.9,2413.0,17.0155,Jumanji,NaN,When siblings Judy and Peter discover an encha...
2,15602,Romance Comedy,6.5,92.0,11.7129,Grumpier Old Men,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",A family wedding reignites the ancient feud be...
3,31357,Comedy Drama Romance,6.1,34.0,3.85949,Waiting to Exhale,NaN,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Comedy,5.7,173.0,8.38752,Father of the Bride Part II,"{'id': 96871, 'name': 'Father of the Bride Col...",Just when George Banks has recovered from his ...


In [73]:
tot_data.shape

(5000, 8)

In [71]:
tot_data=tot_data[0:5000]

In [68]:
tot_data = tot_data[['id','genres','vote_average','vote_count','popularity','title','belongs_to_collection','overview']]

In [64]:
tot_data.head(3)

,id,genres,vote_average,vote_count,popularity,title,belongs_to_collection,overview
0,862,Animation Comedy Family,7.7,5415.0,21.9469,Toy Story,"{'id': 10194, 'name': 'Toy Story Collection', ...","Led by Woody, Andy's toys live happily in his ..."
1,8844,Adventure Fantasy Family,6.9,2413.0,17.0155,Jumanji,NaN,When siblings Judy and Peter discover an encha...
2,15602,Romance Comedy,6.5,92.0,11.7129,Grumpier Old Men,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",A family wedding reignites the ancient feud be...


In [41]:
tot_data[['genres','belongs_to_collection']].head(2)

,genres,belongs_to_collection
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","{'id': 10194, 'name': 'Toy Story Collection', ..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN


In [42]:
import ast as ast
tot_data['genres'] = tot_data['genres'].apply(ast.literal_eval)
#tot_data['belongs_to_collection'] = tot_data['belongs_to_collection'].apply(ast.literal_eval)

In [43]:
tot_data[['genres','belongs_to_collection']].head(5)

,genres,belongs_to_collection
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","{'id': 10194, 'name': 'Toy Story Collection', ..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","{'id': 119050, 'name': 'Grumpy Old Men Collect..."
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN
4,"[{'id': 35, 'name': 'Comedy'}]","{'id': 96871, 'name': 'Father of the Bride Col..."


In [44]:
tot_data['genres'] = tot_data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [75]:
tot_data.head(2)

,id,genres,vote_average,vote_count,popularity,title,belongs_to_collection,overview
0,862,Animation Comedy Family,7.7,5415.0,21.9469,Toy Story,"{'id': 10194, 'name': 'Toy Story Collection', ...","Led by Woody, Andy's toys live happily in his ..."
1,8844,Adventure Fantasy Family,6.9,2413.0,17.0155,Jumanji,NaN,When siblings Judy and Peter discover an encha...


In [49]:
#countvetorizer 구현
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer(ngram_range=(1,3))

In [76]:
c_vector_genres = count_vector.fit_transform(tot_data['genres'])

In [77]:
c_vector_genres.shape

(5000, 999)

In [78]:
#코사인유사도로 추천
from sklearn.metrics.pairwise import cosine_similarity
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]

In [79]:
gerne_c_sim.shape

(5000, 5000)

In [112]:
def get_recommend_movie_list(df, movie_title, top=10):
    target_movie_index = df[df['title'] == movie_title].index.values
    
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    
    sim_index = sim_index[sim_index != target_movie_index]
    
    result = df.iloc[sim_index].sort_values('vote_average', ascending = False)[:10]
    #result = df.iloc[sim_index]
    return result

In [113]:
get_recommend_movie_list(tot_data, movie_title = 'Jumanji')

,id,genres,vote_average,vote_count,popularity,title,belongs_to_collection,overview
4766,671,Adventure Fantasy Family,7.5,7188.0,38.1872,Harry Potter and the Philosopher's Stone,"{'id': 1241, 'name': 'Harry Potter Collection'...",Harry Potter has lived under the stairs at his...
3368,879,Adventure Fantasy Comedy Family,6.7,1563.0,14.5081,Hook,NaN,The boy who wasn't supposed to grow up—Peter P...
1981,11319,Fantasy Family Animation Adventure,6.7,609.0,11.3244,The Rescuers,"{'id': 57971, 'name': 'The Rescuers Collection...",What can two little mice possibly do to save a...
2711,1911,Adventure Fantasy Action,6.4,524.0,10.308,The 13th Warrior,NaN,"In AD 922, Arab courtier, Ahmad Ibn Fadlan acc..."
2034,11976,Adventure Fantasy,6.2,282.0,10.9328,Legend,NaN,Set in a timeless mythical forest inhabited by...
4978,16690,Adventure Fantasy Animation Family,6.1,304.0,8.75977,Return to Never Land,"{'id': 55422, 'name': 'Peter Pan Collection', ...",The classic tale of 'Peter Pan' continues in D...
4490,15138,Adventure Fantasy Comedy Family,5.6,53.0,6.25697,Little Monsters,NaN,A young boy is scared of the monster under his...
3049,47070,Adventure Fantasy,5.1,7.0,6.56171,Brenda Starr,NaN,Mike is a struggling artist who draws the 'Bre...
1741,6264,Adventure Fantasy,4.7,10.0,1.74053,Prince Valiant,NaN,"An Arthurian legend of young Prince Valiant, s..."
